In [6]:
# Dependencies
import json
import requests

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import operator
from pprint import pprint

In [7]:
# Original Dataframe used for Revenue, Budget information
path = "tmdb-5000-movie-dataset/tmdb_5000_movies.csv"
df = pd.read_csv(path)
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [8]:
# New Dataframe used for Analysis
path = "moviesapi.csv"
new_df = pd.read_csv(path)
new_df.head()

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,Metascore,IMDB Rating
0,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,83.0,7.8
1,Pirates of the Caribbean: At World's End,2007,PG-13,169 min,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Ted Elliott (charac...","Mandarin, English",USA,Nominated for 2 Oscars. Another 20 wins & 44 n...,50.0,7.1
2,Spectre,2015,PG-13,148 min,"Action, Adventure, Thriller",Sam Mendes,"John Logan (screenplay by), Neal Purvis (scree...","English, Spanish, Italian, German, French","UK, USA",Won 1 Oscar. Another 7 wins & 32 nominations.,60.0,6.8
3,The Dark Knight Rises,2012,PG-13,164 min,"Action, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","English, Arabic","UK, USA",Nominated for 1 BAFTA Film Award. Another 38 w...,78.0,8.4
4,John Carter,2012,PG-13,132 min,"Action, Adventure, Sci-Fi",Andrew Stanton,"Andrew Stanton (screenplay by), Mark Andrews (...",English,USA,2 wins & 8 nominations.,51.0,6.6


In [9]:
# Movie release dates are between 1916 and 2017
df['release_date'].sort_values(ascending= False).head(5)

4255    2017-02-03
3408    2016-10-02
3302    2016-09-16
4720    2016-09-09
3249    2016-09-09
Name: release_date, dtype: object

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [11]:
df = df.drop(columns= 'tagline')
df = df.drop(columns= 'homepage')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 18 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null int64
dtypes: float64(3), int64(4), object(11)
memory usage: 675.5+ KB


In [12]:
# title_list_0_5000 = list(df['title'][0:])
# #print(title_list_0_5000)

In [10]:
# # Test for one movie

# apikey = "trilogy"
# movie = "jack reacher"
# query_url = f"http://www.omdbapi.com/?apikey={apikey}&"
# # response = requests.get(f'{query_url}t={movie}').json()
# # pprint(response)

In [11]:
# set up lists to hold reponse info

# titles = []
# year = []
# rated = []
# released = []
# runtime = []
# genre = []
# director = []
# writer = []
# language = []
# country = []
# awards = []
# m_ratings = []
# i_ratings = []
# boxoffice = []
# production = []

# record = 0
# loop_set = 1


# # 

# print("Beginning Data Retrieval")
# print("------------------------")

# for title in title_list_0_5000:
#     response = requests.get(f'{query_url}t={title}').json()
#     record += 1
#     #pprint(response)
    
#     try:
#         titles.append(response['Title'])
#         year.append(response['Year'])
#         rated.append(response['Rated'])
#         released.append(response['Released'])
#         runtime.append(response['Runtime'])
#         genre.append(response['Genre'])
#         director.append(response['Director'])
#         writer.append(response['Writer'])
#         language.append(response['Language'])
#         country.append(response['Country'])
#         awards.append(response['Awards'])
#         m_ratings.append(response['Metascore'])
#         i_ratings.append(response['imdbRating'])
#         boxoffice.append(response['BoxOffice'])
#         production.append(response['Production'])
        
        
#         print(f"Processing Record {record} of Set {loop_set}: {title}")
    
#     except(KeyError):
#         print("Title is not found... skipping.")
#         loop_set += 1

# print("------------------------")
# print("Data Retrieval Complete!")
# print("------------------------")

Beginning Data Retrieval
------------------------
Processing Record 1 of Set 1: Avatar
Processing Record 2 of Set 1: Pirates of the Caribbean: At World's End
Processing Record 3 of Set 1: Spectre
Processing Record 4 of Set 1: The Dark Knight Rises
Processing Record 5 of Set 1: John Carter
Processing Record 6 of Set 1: Spider-Man 3
Processing Record 7 of Set 1: Tangled
Processing Record 8 of Set 1: Avengers: Age of Ultron
Processing Record 9 of Set 1: Harry Potter and the Half-Blood Prince
Processing Record 10 of Set 1: Batman v Superman: Dawn of Justice
Processing Record 11 of Set 1: Superman Returns
Processing Record 12 of Set 1: Quantum of Solace
Processing Record 13 of Set 1: Pirates of the Caribbean: Dead Man's Chest
Processing Record 14 of Set 1: The Lone Ranger
Processing Record 15 of Set 1: Man of Steel
Processing Record 16 of Set 1: The Chronicles of Narnia: Prince Caspian
Processing Record 17 of Set 1: The Avengers
Processing Record 18 of Set 1: Pirates of the Caribbean: On St

Processing Record 163 of Set 1: Stealth
Processing Record 164 of Set 1: Watchmen
Processing Record 165 of Set 1: Lethal Weapon 4
Processing Record 166 of Set 1: Hulk
Processing Record 167 of Set 1: G.I. Joe: Retaliation
Processing Record 168 of Set 1: Sahara
Processing Record 169 of Set 1: Final Fantasy: The Spirits Within
Processing Record 170 of Set 1: Captain America: The First Avenger
Processing Record 171 of Set 1: The World Is Not Enough
Processing Record 172 of Set 1: Master and Commander: The Far Side of the World
Processing Record 173 of Set 1: The Twilight Saga: Breaking Dawn - Part 2
Processing Record 174 of Set 1: Happy Feet Two
Processing Record 175 of Set 1: The Incredible Hulk
Processing Record 176 of Set 1: The BFG
Processing Record 177 of Set 1: The Revenant
Processing Record 178 of Set 1: Turbo
Processing Record 179 of Set 1: Rango
Processing Record 180 of Set 1: Penguins of Madagascar
Processing Record 181 of Set 1: The Bourne Ultimatum
Processing Record 182 of Set 1

Processing Record 330 of Set 4: The Lord of the Rings: The Return of the King
Processing Record 331 of Set 4: The Lord of the Rings: The Two Towers
Processing Record 332 of Set 4: Seventh Son
Processing Record 333 of Set 4: Lara Croft: Tomb Raider
Processing Record 334 of Set 4: Transcendence
Processing Record 335 of Set 4: Jurassic Park III
Processing Record 336 of Set 4: Rise of the Planet of the Apes
Processing Record 337 of Set 4: The Spiderwick Chronicles
Processing Record 338 of Set 4: A Good Day to Die Hard
Processing Record 339 of Set 4: The Alamo
Processing Record 340 of Set 4: The Incredibles
Processing Record 341 of Set 4: Cutthroat Island
Title is not found... skipping.
Processing Record 343 of Set 5: Men in Black
Processing Record 344 of Set 5: Toy Story 2
Processing Record 345 of Set 5: Unstoppable
Processing Record 346 of Set 5: Rush Hour 2
Processing Record 347 of Set 5: What Lies Beneath
Processing Record 348 of Set 5: Cloudy with a Chance of Meatballs
Processing Recor

Processing Record 501 of Set 7: 2 Fast 2 Furious
Processing Record 502 of Set 7: The Little Prince
Processing Record 503 of Set 7: The Invasion
Processing Record 504 of Set 7: The Adventures of Rocky & Bullwinkle
Processing Record 505 of Set 7: The Secret Life of Pets
Processing Record 506 of Set 7: The League of Extraordinary Gentlemen
Processing Record 507 of Set 7: Despicable Me 2
Processing Record 508 of Set 7: Independence Day
Processing Record 509 of Set 7: The Lost World: Jurassic Park
Processing Record 510 of Set 7: Madagascar
Processing Record 511 of Set 7: Children of Men
Processing Record 512 of Set 7: X-Men
Processing Record 513 of Set 7: Wanted
Processing Record 514 of Set 7: The Rock
Processing Record 515 of Set 7: Ice Age: The Meltdown
Processing Record 516 of Set 7: 50 First Dates
Processing Record 517 of Set 7: Hairspray
Processing Record 518 of Set 7: Exorcist: The Beginning
Processing Record 519 of Set 7: Inspector Gadget
Processing Record 520 of Set 7: Now You See M

Processing Record 676 of Set 7: Jurassic Park
Processing Record 677 of Set 7: Wyatt Earp
Processing Record 678 of Set 7: Clear and Present Danger
Processing Record 679 of Set 7: Dragon Blade
Processing Record 680 of Set 7: Little Man
Processing Record 681 of Set 7: U-571
Processing Record 682 of Set 7: The American President
Processing Record 683 of Set 7: The Love Guru
Processing Record 684 of Set 7: 3000 Miles to Graceland
Processing Record 685 of Set 7: The Hateful Eight
Processing Record 686 of Set 7: Blades of Glory
Processing Record 687 of Set 7: Hop
Processing Record 688 of Set 7: 300
Processing Record 689 of Set 7: Meet the Fockers
Processing Record 690 of Set 7: Marley & Me
Processing Record 691 of Set 7: The Green Mile
Processing Record 692 of Set 7: Wild Hogs
Processing Record 693 of Set 7: Chicken Little
Processing Record 694 of Set 7: Gone Girl
Processing Record 695 of Set 7: The Bourne Identity
Processing Record 696 of Set 7: GoldenEye
Processing Record 697 of Set 7: The 

...

Processing Record 4026 of Set 78: Religulous
Processing Record 4027 of Set 78: Fuel
Processing Record 4028 of Set 78: Valley of the Heart's Delight
Processing Record 4029 of Set 78: Eye of the Dolphin
Processing Record 4030 of Set 78: 8: The Mormon Proposition
Processing Record 4031 of Set 78: The Other End of the Line
Processing Record 4032 of Set 78: Anatomy
Processing Record 4033 of Set 78: Sleep Dealer
Processing Record 4034 of Set 78: Super
Processing Record 4035 of Set 78: Christmas Mail
Processing Record 4036 of Set 78: Stung
Processing Record 4037 of Set 78: Antibirth
Processing Record 4038 of Set 78: Get on the Bus
Processing Record 4039 of Set 78: Thr3e
Processing Record 4040 of Set 78: Idiocracy
Processing Record 4041 of Set 78: The Rise of the Krays
Processing Record 4042 of Set 78: This Is England
Processing Record 4043 of Set 78: U.F.O.
Processing Record 4044 of Set 78: Bathing Beauty
Processing Record 4045 of Set 78: Go for It!
Processing Record 4046 of Set 78: Dancer, T

Processing Record 4191 of Set 89: May
Processing Record 4192 of Set 89: Against the Wild
Processing Record 4193 of Set 89: Under the Same Moon
Processing Record 4194 of Set 89: Conquest of the Planet of the Apes
Processing Record 4195 of Set 89: In the Bedroom
Processing Record 4196 of Set 89: I Spit on Your Grave
Processing Record 4197 of Set 89: Happy, Texas
Processing Record 4198 of Set 89: My Summer of Love
Processing Record 4199 of Set 89: The Lunchbox
Processing Record 4200 of Set 89: Yes
Processing Record 4201 of Set 89: You Can't Take It With You
Processing Record 4202 of Set 89: From Here to Eternity
Processing Record 4203 of Set 89: She Wore a Yellow Ribbon
Processing Record 4204 of Set 89: Grace Unplugged
Processing Record 4205 of Set 89: Foolish
Processing Record 4206 of Set 89: N-Secure
Processing Record 4207 of Set 89: Caramel
Processing Record 4208 of Set 89: Out of the Dark
Processing Record 4209 of Set 89: The Bubble
Processing Record 4210 of Set 89: The Conversation
P

Processing Record 4354 of Set 93: The Green Inferno
Processing Record 4355 of Set 93: Departure
Processing Record 4356 of Set 93: The Sessions
Processing Record 4357 of Set 93: Food, Inc.
Processing Record 4358 of Set 93: October Baby
Processing Record 4359 of Set 93: Next Stop Wonderland
Processing Record 4360 of Set 93: The Skeleton Twins
Processing Record 4361 of Set 93: Martha Marcy May Marlene
Processing Record 4362 of Set 93: Obvious Child
Processing Record 4363 of Set 93: Frozen River
Processing Record 4364 of Set 93: 20 Feet from Stardom
Processing Record 4365 of Set 93: Two Girls and a Guy
Processing Record 4366 of Set 93: Walking and Talking
Processing Record 4367 of Set 93: Who Killed the Electric Car?
Processing Record 4368 of Set 93: The Broken Hearts Club: A Romantic Comedy
Processing Record 4369 of Set 93: Bubba Ho-tep
Processing Record 4370 of Set 93: Slam
Processing Record 4371 of Set 93: Brigham City
Processing Record 4372 of Set 93: Fiza
Processing Record 4373 of Set

Processing Record 4518 of Set 96: Mad Hot Ballroom
Processing Record 4519 of Set 96: The Station Agent
Processing Record 4520 of Set 96: To Save A Life
Processing Record 4521 of Set 96: Wordplay
Processing Record 4522 of Set 96: Beyond the Mat
Processing Record 4523 of Set 96: The Singles Ward
Processing Record 4524 of Set 96: Osama
Processing Record 4525 of Set 96: Sholem Aleichem: Laughing In The Darkness
Processing Record 4526 of Set 96: Groove
Processing Record 4527 of Set 96: The R.M.
Processing Record 4528 of Set 96: Twin Falls Idaho
Processing Record 4529 of Set 96: Mean Creek
Processing Record 4530 of Set 96: Hurricane Streets
Processing Record 4531 of Set 96: Never Again
Processing Record 4532 of Set 96: Civil Brand
Processing Record 4533 of Set 96: Lonesome Jim
Processing Record 4534 of Set 96: Drinking Buddies
Processing Record 4535 of Set 96: Deceptive Practice: The Mysteries and Mentors of Ricky Jay
Processing Record 4536 of Set 96: Seven Samurai
Processing Record 4537 of 

Processing Record 4680 of Set 101: Call + Response
Processing Record 4681 of Set 101: Malevolence
Title is not found... skipping.
Processing Record 4683 of Set 102: Super Hybrid
Processing Record 4684 of Set 102: Baghead
Processing Record 4685 of Set 102: American Beast
Processing Record 4686 of Set 102: The Case of the Grinning Cat
Processing Record 4687 of Set 102: Ordet
Processing Record 4688 of Set 102: Good Dick
Processing Record 4689 of Set 102: The Man from Earth
Processing Record 4690 of Set 102: The Trials Of Darryl Hunt
Title is not found... skipping.
Processing Record 4692 of Set 103: Yesterday Was a Lie
Processing Record 4693 of Set 103: Theresa Is a Mother
Processing Record 4694 of Set 103: H.
Processing Record 4695 of Set 103: Archaeology of a Woman
Processing Record 4696 of Set 103: Children of Heaven
Processing Record 4697 of Set 103: Weekend
Processing Record 4698 of Set 103: She's Gotta Have It
Processing Record 4699 of Set 103: Butterfly Girl
Processing Record 4700 o

In [19]:
# movies_df = pd.DataFrame({"Title" : titles, 
#                           "Year" : year, 
#                           "Rated" : rated, 
#                           "Runtime" : runtime, 
#                           "Genre" : genre,
#                           "Director" : director,
#                           "Writer" : writer,
#                           "Language" : language,
#                           "Country" : country,
#                           "Awards" : awards,
#                           "Metascore" : m_ratings,
#                           "IMDB Rating" : i_ratings})
# #                          "Production Company" : production})
# #                          "Box Office" : boxoffice})


# movies_df.to_csv("moviesapi.csv", header= True, index= False)
# movies_df.head()

,Title,Year,Rated,Runtime,Genre,Director,Writer,Language,Country,Awards,Metascore,IMDB Rating
0,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"English, Spanish","UK, USA",Won 3 Oscars. Another 85 wins & 128 nominations.,83,7.8
1,Pirates of the Caribbean: At World's End,2007,PG-13,169 min,"Action, Adventure, Fantasy",Gore Verbinski,"Ted Elliott, Terry Rossio, Ted Elliott (charac...","Mandarin, English",USA,Nominated for 2 Oscars. Another 20 wins & 44 n...,50,7.1
2,Spectre,2015,PG-13,148 min,"Action, Adventure, Thriller",Sam Mendes,"John Logan (screenplay by), Neal Purvis (scree...","English, Spanish, Italian, German, French","UK, USA",Won 1 Oscar. Another 7 wins & 32 nominations.,60,6.8
3,The Dark Knight Rises,2012,PG-13,164 min,"Action, Thriller",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","English, Arabic","UK, USA",Nominated for 1 BAFTA Film Award. Another 38 w...,78,8.4
4,John Carter,2012,PG-13,132 min,"Action, Adventure, Sci-Fi",Andrew Stanton,"Andrew Stanton (screenplay by), Mark Andrews (...",English,USA,2 wins & 8 nominations.,51,6.6


In [13]:
# Highest revenue generating movies 
revenue_df = pd.DataFrame({'Title' : df['title'],
                           'Revenue': df['revenue'],
                           'Budget' : df['budget']})
revenue_df.sort_values(by= 'Revenue', ascending=False).head()

,Title,Revenue,Budget
0,Avatar,2787965087,237000000
25,Titanic,1845034188,200000000
16,The Avengers,1519557910,220000000
28,Jurassic World,1513528810,150000000
44,Furious 7,1506249360,190000000


In [ ]:
# Bubble plot to show revenue by year, with bubble size corresponding to score
